In [1]:
from MultiEurlexDataset import MultiEurlexDataset
from MultiEvalDataset import MultiEvalDataset
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline, EarlyStoppingCallback
from torch.utils.data import RandomSampler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import numpy as np
import torch.nn as nn

In [2]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
#tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=False, trunaction=False)
#training_dataset = MultiEvalDataset(languages = ['hr'])
#lengths = []
#for i in range(len(training_dataset)):
#    lengths.append(len(training_dataset[i][0].split(' ')))
#for i in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
#    print(np.percentile(lengths, i, axis=0))

In [4]:
device='cuda'
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=True, trunaction=True)
training_dataset = MultiEurlexDataset(languages = ['en'], tokenizer=tokenizer)
eval_dataset = MultiEurlexDataset(split='validation',languages = ['hr'], tokenizer=tokenizer)

Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


In [5]:
test_dataset = MultiEurlexDataset(split='test',languages = ['hr'], tokenizer=tokenizer)

Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


In [6]:
collator_fn = DataCollatorWithPadding(tokenizer,return_tensors="pt", padding="max_length", max_length=512)

In [7]:
metric = f1_score

def compute_metrics(eval_pred):
    label_pred, label_true = eval_pred
    sigmoid = nn.Sigmoid()
    label_pred=sigmoid(torch.tensor(label_pred)).numpy()
    label_pred[label_pred < 0.5] = 0
    label_pred[label_pred >= 0.5] = 1
    result={}
    result['f1'] = metric(label_true,label_pred, average='samples')
    return result

In [8]:
training_args = TrainingArguments(
    output_dir='./results/germanic',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_ratio=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_strategy='steps',
    logging_steps=1000,
    learning_rate=1e-5,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    lr_scheduler_type='polynomial',
    log_level='info',
    dataloader_num_workers=8
)

In [9]:
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=21, problem_type="multi_label_classification", )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    
    compute_metrics=compute_metrics,
    data_collator=collator_fn,
    #callbacks=[
    #    EarlyStoppingCallback(early_stopping_patience=5),
    #]
)

In [11]:
training_args.num_train_epochs

10

In [12]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, function_to_apply='sigmoid')

In [13]:
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

/home/mrajnovic/miniconda3/envs/tar/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 55000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 68750


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 5000
  Batch size = 16


PredictionOutput(predictions=array([[-3.169377  , -4.543096  , -3.7680247 , ...,  0.7034655 ,
        -2.6702948 , -3.112781  ],
       [-2.285403  , -3.7878382 , -4.3809805 , ...,  2.5344818 ,
        -3.963364  , -4.3044024 ],
       [-2.0230477 , -3.7235363 , -3.9355998 , ..., -1.0295571 ,
        -3.3179023 , -2.5032394 ],
       ...,
       [-4.177247  , -0.31297272, -3.3858912 , ..., -2.6987107 ,
        -3.5008006 , -3.5991797 ],
       [-1.1728051 , -2.9491198 , -2.7745988 , ...,  0.54032177,
        -2.884429  , -2.394803  ],
       [-4.5849667 , -5.0209565 , -1.4495361 , ..., -1.1971147 ,
        -3.9299302 , -5.317438  ]], dtype=float32), label_ids=array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32), metrics={'test_loss': 0.2505229711532593, 'test_f1': 0.6447666089466089, 't

In [ ]:
model(input_ids=training_dataset[0]['input_ids'].unsqueeze(0).to(device),attention_mask=training_dataset[0]['attention_mask'].unsqueeze(0).to(device))

SequenceClassifierOutput(loss=None, logits=tensor([[-1.2594,  2.0396, -3.9613,  1.3616, -2.1823, -3.1136, -3.2287,  0.2411,
         -2.1513, -3.2607, -3.1584, -2.7092, -0.9786, -3.5652, -1.6449, -1.1268,
         -1.3294, -3.5044, -3.3167, -3.3110,  0.1766]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)